In [20]:
import pandas as pd
from bs4 import BeautifulSoup

In [21]:
df=pd.read_csv('train.csv')

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60115 entries, 0 to 60114
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Sentence_id   60115 non-null  object
 1   New_Sentence  60115 non-null  object
 2   Type          60115 non-null  object
dtypes: object(3)
memory usage: 1.4+ MB


In [23]:
df['Type'].value_counts()

Type
Responsibility    15561
Requirement       14132
SoftSkill          9581
Experience         9248
Skill              6956
Education          4637
Name: count, dtype: int64

In [24]:
df=df.groupby('Type').apply(lambda group: group.head(4500)).reset_index(drop=True)

C:\Users\YB628LA\AppData\Local\Temp\ipykernel_38516\1758705373.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df=df.groupby('Type').apply(lambda group: group.head(4500)).reset_index(drop=True)


In [25]:
df['Type'].value_counts()

Type
Education         4500
Experience        4500
Requirement       4500
Responsibility    4500
Skill             4500
SoftSkill         4500
Name: count, dtype: int64

In [26]:
df.isnull().sum()

Sentence_id     0
New_Sentence    0
Type            0
dtype: int64

In [27]:
df.duplicated().sum()

0

In [28]:
def transform_sentence(html_string):
    soup = BeautifulSoup(html_string, 'html.parser')
    input_string= soup.get_text()
    input_string = input_string.replace("b'", "")
    input_string = ' '.join([word for word in input_string.split() if not word.startswith('\\\\')])
    input_string.lower()
    return input_string

In [29]:
df['New_Sentence']=df['New_Sentence'].apply(lambda x: transform_sentence(x))


In [30]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
sw_list=stopwords.words('english')
df['New_Sentence'] = df['New_Sentence'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\YB628LA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [31]:
X=df['New_Sentence']
y=df['Type']

In [32]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

y = label_encoder.fit_transform(y)

In [33]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [35]:
tfidf = TfidfVectorizer()

In [36]:
X_train_tfidf = tfidf.fit_transform(X_train).toarray()
X_test_tfidf = tfidf.transform(X_test)

In [43]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [44]:
model = RandomForestClassifier()
model.fit(X_train_tfidf,y_train)
y_pred = model.predict(X_test_tfidf)

In [45]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.76      0.87      0.81       926
           1       0.95      0.88      0.92       919
           2       0.60      0.29      0.39       923
           3       0.74      0.84      0.79       873
           4       0.57      0.69      0.63       865
           5       0.67      0.76      0.71       894

    accuracy                           0.72      5400
   macro avg       0.72      0.72      0.71      5400
weighted avg       0.72      0.72      0.71      5400



In [49]:
df_test= pd.read_csv('test.csv')

In [50]:
df_test

,Sentence_id,New_Sentence
0,UAERES16346,<html> b'Collaborate across all of DB&T practi...
1,COGREQ15586,b'Strong \\u2705 technology expertise in Ident...
2,UAEREQ12722,b'Strong knowledge <br> on Service \\U0001f517...
3,COGSKL29155,<html> b'Architect scalable data processing an...
4,PHERES12551,"<html> b'Map client organization, build outsta..."
...,...,...
15024,HONEXP17084,<html> b'A minimum <p> of 3 </p> years of expe...
15025,UAESSK45553,<html> b'Track record of managing multiple pro...
15026,GERSSK14317,"<html> b'In addition, the candidate should hav..."
15027,PHEREQ4980,b'Candidate should <h1> have sufficient </h1> ...


In [51]:
df_test['New_Sentence']=df_test['New_Sentence'].apply(lambda x: transform_sentence(x))

In [52]:
df_test['New_Sentence'] = df_test['New_Sentence'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

In [53]:
X_test_final=df_test['New_Sentence']

In [55]:
X_test_final_tfidf = tfidf.transform(X_test_final)

In [57]:
y_pred_final = model.predict(X_test_final_tfidf)

In [58]:
y_pred_final

array([3, 5, 4, ..., 2, 4, 4])

In [63]:
type=label_encoder.inverse_transform(y_pred_final)

In [68]:
df_final=pd.DataFrame({'Sentence_id': df_test['Sentence_id'], 'Type': type})

In [69]:
df_final.head()

,Sentence_id,Type
0,UAERES16346,Responsibility
1,COGREQ15586,SoftSkill
2,UAEREQ12722,Skill
3,COGSKL29155,Skill
4,PHERES12551,Responsibility


In [71]:
df_final.to_csv('submission.csv', index=False)